In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
def get_heroes_list():
    heroes_list_url = 'https://dota2protracker.com/'
    response = requests.get(heroes_list_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    href_tags = soup.find_all(href=True)
    hero_links = soup.select('table a')
    return {hero.text: hero['href'] for hero in hero_links}

def parse_hero_build(hero_url):
    response = requests.get(hero_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    builds = {}
    positions = soup.select('.position-panel')

    for position in positions:
        position_name = position.select_one('.panel-title').text.strip()
        builds[position_name] = {}
        facets = position.select('.facet-panel')

        for facet in facets:
            facet_name = facet.select_one('.panel-title').text.strip()
            builds[position_name][facet_name] = []
            build_links = facet.select('a')
            
            for link in build_links:
                build_url = link['href']
                builds[position_name][facet_name].append(parse_build(build_url))

    return builds

def parse_build(build_url):
    response = requests.get(build_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    build_data = {
        'Abilities': [],
        'Starting Items': [],
        'Common Late Game Inventory': [],
        'Early Game (0-12)': [],
        'Mid Game (12-35)': [],
        'Late Game (35+)': [],
        'Core Build': []
    }

    sections = soup.select('.build-section')

    for section in sections:
        section_title = section.select_one('.section-title').text.strip()
        if section_title in build_data:
            items = section.select('.item')
            build_data[section_title] = [item.text for item in items]

    return build_data

In [3]:
heroes = get_heroes_list()
heroes

{}

In [4]:
all_builds = {}

for hero, url in heroes.items():
    all_builds[hero] = parse_hero_build(url)

all_builds

{}

In [5]:
with open('dota2_builds.json', 'w') as f:
    json.dump(all_builds, f, indent=4)